# Polígonos

Descarga polígonos para cada manzano a nivel nacional del geoportal del INE.

In [87]:
import requests
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
from io import BytesIO
from openpyxl import load_workbook
import json
from IPython.display import clear_output
from pathlib import Path

( Una lista de municipios tomada de microdatos del censo 2024 )

In [51]:
municipios = pd.read_csv("recursos/municipios.csv", index_col=["codigo"])
municipios

,municipio,departamento,poblacion
codigo,,,
10101,Sucre,Chuquisaca,296746
10102,Yotala,Chuquisaca,9256
10103,Poroma,Chuquisaca,13649
10201,Azurduy,Chuquisaca,7903
10202,Tarvita,Chuquisaca,16812
...,...,...,...
90401,Santa Rosa,Pando,2889
90402,Ingavi,Pando,2600
90501,Nueva Esperanza,Pando,1598


Cabezales para cada consulta.

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:144.0) Gecko/20100101 Firefox/144.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Content-Type': 'application/json',
    'Origin': 'https://geoportal.ine.gob.bo',
    'Sec-GPC': '1',
    'Connection': 'keep-alive',
    'Referer': 'https://geoportal.ine.gob.bo/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
}

In [3]:
def refresh_headers():
    response = requests.post(
        "https://wgeoportal.ine.gob.bo/api/v1/geoportal/registroSesion",
    )
    r = response.json()
    token = r["session_token"]
    headers["Authorization"] = f"Bearer {token}"
    headers["X-Session-Token"] = token

Descargar polígonos para un municipio.

In [4]:
def get_manzanos_municipio(codigo):
    response = requests.post(
        "https://wgeoportal.ine.gob.bo/api/v1/selecccion/depMunSeleccionadoPoligono",
        headers=headers,
        json={
            "id": "0" + str(codigo),
            "tipo": "municipio",
        },
    )

    while True:
        if response.status_code == 200:
            municipio = municipios.loc[codigo][["departamento", "municipio"]].to_dict()

            features = [
                {
                    "type": "Feature",
                    "properties": {
                        **municipio,
                        **{key: feature[key] for key in ["codigo", "nombre"]},
                    },
                    "geometry": feature["geojson"],
                }
                for feature in response.json()
            ]

            return features
        else:
            refresh_headers()

Descargar polígonos para todos los municipios.

In [5]:
def get_manzanos():
    features = []
    for codigo in tqdm(municipios.index):
        features.extend(get_manzanos_municipio(codigo))
    gdf = gpd.GeoDataFrame.from_features({
        "type": "FeatureCollection",
        "features": features
    })
    return gdf[["departamento", "municipio", "nombre", "codigo", "geometry"]]

Inicializar cabezales y descargar!

In [6]:
refresh_headers()

In [60]:
gdf = get_manzanos()

100%|██████████| 342/342 [03:06<00:00,  1.83it/s]


In [84]:
gdf

,departamento,municipio,nombre,codigo,geometry
0,Chuquisaca,Sucre,AZARI BAJA,04141933837-A,"MULTIPOLYGON (((-65.24107 -19.10018, -65.24147..."
1,Chuquisaca,Sucre,AZARI BAJA,04143579299-A,"MULTIPOLYGON (((-65.24233 -19.09727, -65.2423 ..."
2,Chuquisaca,Sucre,AZARI BAJA,04140381143-A,"MULTIPOLYGON (((-65.23402 -19.10458, -65.23388..."
3,Chuquisaca,Sucre,AZARI BAJA,04143356216-A,"MULTIPOLYGON (((-65.23883 -19.09875, -65.23848..."
4,Chuquisaca,Sucre,AZARI BAJA,04139661122-A,"MULTIPOLYGON (((-65.23175 -19.10594, -65.23118..."
...,...,...,...,...,...
247341,Pando,Santos Mercado,RESERVA II,11593459076-A,"MULTIPOLYGON (((-66.16147 -10.52149, -66.16137..."
247342,Pando,Santos Mercado,RESERVA II,11593377823-A,"MULTIPOLYGON (((-66.15746 -10.52083, -66.15746..."
247343,Pando,Santos Mercado,RESERVA II,11592872074-A,"MULTIPOLYGON (((-66.15968 -10.52156, -66.15969..."
247344,Pando,Santos Mercado,RESERVA II,11592118798-A,"MULTIPOLYGON (((-66.15829 -10.52165, -66.1583 ..."


Guardar como geoparquet.

In [62]:
gdf.to_parquet("datos/manzanos.parquet")